# Using asistem to analyse BF images with magnetic contrast

The asistem module was developed to analyse four images captured with the electron beam displaced to west, south, east and north edges of the ADF-detector.

In [1]:
%matplotlib qt

import hyperspy.api as hs
import numpy as np
import imageio
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pyxem.utils.pixelated_stem_tools as pst


import asistem.image_tools as aipit
import asistem.pattern_fitting as aippf
import asistem.magnetic_analysis_tools as aipmat

In [11]:
# The rotation of the sample relative to the detector
rotation = 113

# Load the four images
signals = hs.load('*.dm3')
s_west  = signals[3]
s_south = signals[2]
s_east  = signals[0]
s_north = signals[1]

# Set the scale for the scalebar
scale = s_west.axes_manager[0].scale

In [3]:
# Import the pattern to use as a mask
pattern = imageio.imread('pinwheelASI.png')

#### Leveling the intensity of the images

In [4]:
sw = aipit.level_intensity(s_west)
se = aipit.level_intensity(s_east)
ss = aipit.level_intensity(s_south)
sn = aipit.level_intensity(s_north)

#### Recreating the BF image and find corners of the FIB-milled pattern

In [5]:
sss = aipit.recreate_bf_image(ss,sn,se,sw)

[########################################] | 100% Completed |  0.1s


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
approximate_pattern = aipit.compute_approximate_pattern(sss)

In [7]:
coords = aippf.compute_corners(approximate_pattern, k=0.003, sigma=24, plot=True)
assert coords.shape[0] == 4

In [8]:
mask = aippf.maximise_pattern_fit(sss, pattern, coords, g=.15)

Initial std: 66.308
Goal: 56.361
Still working, but found 60.317
Still working, but found 59.885
Still working, but found 59.661
Still working, but found 59.206
Still working, but found 57.16
Still working, but found 56.488
Still working, but found 56.4
Finished after 102.214s,
with std = 54.631.


### Calculating the DPC-image and saving the images and magnetic data

In [9]:
m_s, s = aipit.calculate_dpc_image(ss, sn, sw, se, mask, coords=coords)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [12]:
# Show the DPC x and y signals, along with the DPC colour plots
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
sigma = 3.5

axx   = axs[0, 0]
axy   = axs[0, 1]
axc   = axs[1, 0]
m_axc = axs[1, 1]

sb_dpcx = ScaleBar(scale, 'um', location='lower right', color='white',
                    frameon=False, width_fraction=.02, label_formatter = lambda x, y:'')
sb_dpcy = ScaleBar(scale, 'um', location='lower right', color='white', 
                    frameon=False, width_fraction=.02, label_formatter = lambda x, y:'')

ax_inset = inset_axes(axc, width="15%", height="15%",
                      bbox_to_anchor=(0.,0.,1.,1.), bbox_transform=axc.transAxes)
m_ax_inset = inset_axes(m_axc, width="15%", height="15%",
                      bbox_to_anchor=(0.,0.,1.,1.), bbox_transform=m_axc.transAxes)

axx.imshow(s.inav[0].data, cmap='viridis')
axx.add_artist(sb_dpcx)
axy.imshow(s.inav[1].data, cmap='viridis')
axy.add_artist(sb_dpcy)
s.get_color_image_with_indicator(ax=axc, ax_indicator=ax_inset, autolim_sigma=4,
                                 indicator_rotation=rotation)
m_s.get_color_image_with_indicator(ax=m_axc, ax_indicator=m_ax_inset, indicator_rotation=rotation,
                                       autolim_sigma=sigma)

axx.set_title('DPC x')
axy.set_title('DPC y')
axc.set_title('DPC color')
m_axc.set_title('DPC color of masked signal')

for ax in fig.axes:
    ax.set_axis_off()

fig.tight_layout()
# fig.savefig('filename.png', dpi=300)

C:\Users\Bruker\AppData\Local\Temp/ipykernel_37720/1417583124.py:37: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


#### Do the segmentation and extract info from masked ASI

In [13]:
seg1 = aipmat.get_segmentation_of_dpc_image(m_s, plot=False)

In [14]:
sorted_array_of_magnets = aipmat.extract_magnet_information(seg1, m_s)

In [15]:
tilty = 0
tiltx = -6.9 # This could also be read automatically from the filename
# To save the information [scan_rotation, pattern_rotation, (tilt_info)]
# Pattern rotation is 0 for square ASI and 45 for pinwheel ASI
asi_info = np.array([rotation, 45, (tiltx, tilty)], dtype=object)
np.savez('filename.npz', asi=asi_info, magnets=sorted_array_of_magnets)

## Magnetic analysis of .npz-file

In [16]:
data = np.load('filename.npz', allow_pickle=True, fix_imports=True)

In [17]:
asiinfo = data['asi']
magnets = data['magnets']

In [18]:
arrows, points, approx_macrospin, points_fixed, colours = aipmat.analyse_artificial_spin_ice(magnets, asiinfo, 
                                                                                             variance_threshold=0.03, 
                                                                                             angle_threshold=50)

100%|██████████████████████████████████████████████████████████████████████████████| 220/220 [00:00<00:00, 2350.46it/s]


In [19]:
fig1, ax = plt.subplots(nrows=1, ncols=2, figsize=(14,7))
mins = np.min(arrows, axis=0)
maxs = np.max(arrows, axis=0)
for a in arrows:
    if not (a[0] in points[:,0] and a[1] in points[:,1]):
        ax[0].arrow(a[0], a[1], a[2]/2, a[3]/2, color='salmon', head_width=10, length_includes_head=True)
ax[0].set_xlim((mins[0]-150,maxs[0]+100))
ax[0].set_ylim((maxs[1]+150,mins[1]-150))
ax[0].scatter(points[:,0], points[:,1], c='maroon')

for i, a in enumerate(approx_macrospin):
    ax[1].arrow(a[0], a[1], a[2]/1.1, a[3]/1.1, color=colours[i], head_width=.25, length_includes_head=True)
ax[1].scatter(points_fixed[:,0], points_fixed[:,1], color='k')

ax[1].set_xlim((-2,22))
ax[1].set_ylim((22,-2))

for a in ax:
    a.set_axis_off()
plt.tight_layout()